In [1]:
import tensorflow as tf

# ResNet
![ResNet](https://www.researchgate.net/publication/349646156/figure/fig4/AS:995806349897731@1614430143429/The-architecture-of-ResNet-50-vd-a-Stem-block-b-Stage1-Block1-c-Stage1-Block2.png)


![ResNEt](ResNet.png)
*Implemented a scalable version so can do ResNet-18, ResNet-50 and ResNet-152

In [2]:
class SmallBlock(tf.keras.Model):

    def __init__(self, filter, reduce=False) -> None:
        super().__init__()
        self.stride = 1
        # implement a block with two layers and a residual connection
        if reduce:
            self.stride = 2
            self.conv1x1 = tf.keras.layers.Conv2D(filters=filter, kernel_size=1, strides=self.stride)
        self.conv1 = tf.keras.layers.Conv2D(filters=filter, kernel_size=3, strides=self.stride, padding="same", activation="relu")
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(filters=filter, kernel_size=3, strides=1, padding="same", activation="relu")
        self.bn2 = tf.keras.layers.BatchNormalization()

        self.relu = tf.keras.layers.ReLU()

    def __call__(self, inputs, training=False):
        # reduce size by half if skip connection connects with less filters
        if (self.stride == 2):
            residual = self.conv1x1(inputs)
        else:
            residual = inputs
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        output = tf.keras.layers.add([x, residual])
        return self.relu(output)

class BottleneckBlock(tf.keras.Model):

    def __init__(self, filters, reduce=False) -> None:
        super().__init__()
        stride = 2 if reduce else 1
        
        self.conv1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=1, strides=self.stride, activation="relu")
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, padding="same", activation="relu")
        self.bn2 = tf.keras.layers.BatchNormalization()

        self.conv3 = tf.keras.layers.Conv2D(filters=filters * 4, kernel_size=1, strides=1, activation="relu")
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.relu = tf.keras.layers.ReLU()
 
        self.conv1x1 = tf.keras.layers.Conv2d(filters=filters, kernel_size=1, strides=stride)
        self.relu = tf.keras.layers.ReLU()

    def __call__(self, inputs, training=False):
        residual = inputs

        x = self.conv1(inputs)
        x = self.bn1(x)

        x = self.conv1(x)
        x = self.bn2(x)

        x = self.conv3(x)
        x = self.bn3(x)

        x = self.conv1x1(x)
        output = tf.keras.layers.add([x + residual])

        return self.relu(output)


class ResNet(tf.keras.Model):
    
    def __init__(self, bottleneck = True ,layers = [4, 4, 4, 4]):
        
        super(ResNet, self).__init__()

        if not bottleneck:
            resnet = [tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu")]
            filter = 64
            reduce = False
            for layer in layers:
                for _ in range(layer):
                    resnet.append(SmallBlock(filter, reduce))
                    reduce = False
                filter = filter * 2
                reduce = True
            
            resnet.append(tf.keras.layers.AvgPool2D(pool_size=3, strides=1))
            self.resnet = resnet
            self.classifier = tf.keras.Sequential([
                tf.keras.layers.Dense(units=2048, activation="relu"),
                tf.keras.layers.Dense(units=1024, activation="relu"),
                tf.keras.layers.Dense(units=10, activation="softmax")
            ])
        else:
            # layers greater than 34
            resnet = [tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu")]
            reduce=False
            in_channel = 64
            first_channel = 64
            out_channel = 256
            for layer in layers:
                for _ in range(layer):
                    resnet.append(BottleneckBlock(first_channel ,in_channel, out_channel, reduce=reduce))
                    first_channel = out_channel
                    reduce=False
                reduce=True
                in_channel = in_channel * 2
                out_channel = out_channel * 2
            
            # resnet.append(tf.keras.layers.AvgPool2D(pool_size=3, strides=1))
            self.resnet = resnet
            self.classifier = tf.keras.layers.Sequential([
                tf.keras.layers.Dense(units=4096, activation="relu"),
                tf.keras.layers.Dense(units=2048, activation="relu"),
                tf.keras.layers.Dense(units=10, activation="softmax")
            ])


    def __call__(self, inputs, training=False):
        x = inputs["image"]
        for block in self.resnet:
            x = block(x)
        x = tf.keras.layers.Flatten()(x)
        logits = self.classifier(x)
        return logits
        


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [4]:
model = ResNet(bottleneck=False, layers=[3, 4, 6, 3])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4), loss=["categorical_crossentropy"], metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit({
    "image": x_train
}, y_train, epochs=10)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-11-12 22:24:00.978313: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-12 22:24:00.978596: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2022-11-12 22:24:01.890521: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-12 22:24:04.470062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 439s 278ms/step - loss: 2.4122 - mean_absolute_error: 0.1442 - accuracy: 0.3949
Epoch 2/10
 573/1563 [=========>....................] - ETA: 5:40 - loss: 1.2624 - mean_absolute_error: 0.1178 - accuracy: 0.5401

: 

: 